In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

In [ ]:
# import datasets
train_file = '../input/tabular-playground-series-nov-2021/train.csv'
test_file = '../input/tabular-playground-series-nov-2021/test.csv'
sub_file = '../input/tabular-playground-series-nov-2021/sample_submission.csv'

In [ ]:


train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
submission = pd.read_csv(sub_file)



# Preparation

In [ ]:
train_df.head()

# EDA

In [ ]:
print(f'Number of rows: {train_df.shape[0]}')
print(f'Number of columns: {train_df.shape[1]}')
print(f'No of missing values: {sum(train_df.isna().sum())}')

In [ ]:
train_df.describe()

In [ ]:
test_df.head()

In [ ]:
submission.head()

# Features

In [ ]:
train_df.loc[:, 'f0':'f99'].describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .background_gradient(subset=['std'], cmap='Greens')\
                            .background_gradient(subset=['25%'], cmap='Spectral')\
                            .background_gradient(subset=['50%'], cmap='seismic')\
                            .background_gradient(subset=['75%'], cmap='viridis')\
                            .background_gradient(subset=['mean'], cmap='cubehelix')\
                            .background_gradient(subset=['min'], cmap='Reds')\
                            .background_gradient(subset=['max'], cmap='Blues')

In [ ]:
missing_train_df = pd.DataFrame(train_df.isna().sum())
missing_train_df = missing_train_df.drop(['id', 'target']).reset_index()
missing_train_df.columns = ['feature', 'count']

missing_train_percent_df = missing_train_df.copy()
missing_train_percent_df['count'] = missing_train_df['count']/train_df.shape[0]

missing_test_df = pd.DataFrame(test_df.isna().sum())
missing_test_df = missing_test_df.drop(['id']).reset_index()
missing_test_df.columns = ['feature', 'count']

missing_test_percent_df = missing_test_df.copy()
missing_test_percent_df['count'] = missing_test_df['count']/test_df.shape[0]

features = [feature for feature in train_df.columns if feature not in ['id', 'target']]
missing_train_row = train_df[features].isna().sum(axis=1)
missing_train_row = pd.DataFrame(missing_train_row.value_counts()/train_df.shape[0]).reset_index()
missing_train_row.columns = ['no', 'count']

missing_test_row = test_df[features].isna().sum(axis=1)
missing_test_row = pd.DataFrame(missing_test_row.value_counts()/test_df.shape[0]).reset_index()
missing_test_row.columns = ['no', 'count']

In [ ]:
cat_features =[]
num_features =[]

for col in train_df.columns:
    if train_df[col].dtype=='float64':
        num_features.append(col)
    else:
        cat_features.append(col)
print('Catagoric features: ', cat_features)
display(len(cat_features))
print('Numerical features: ', num_features)
display(len(num_features))

# Modeling

In [ ]:
#Drop the Target Column
training_data = train_df.drop("target", axis=1)
#Save the value of Target for usage
training_label = train_df["target"].copy()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
lgbm_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy="most_frequent")),
    ('std_scaler', StandardScaler()),
])
training_prepared = lgbm_pipeline.fit_transform(training_data)

In [ ]:
import lightgbm as lgb
lgbm_cls = lgb.LGBMClassifier()
lgbm_cls.fit(training_prepared,training_label)

In [ ]:
testing_prepared = lgbm_pipeline.fit_transform(test_df)
test_predictions = lgbm_cls.predict(testing_prepared)

In [ ]:
submission = pd.read_csv(sub_file)
submission['target'] = list(map(float, test_predictions))
submission.to_csv('submission.csv', index=False)